# SQL Database Generation by Wesley Giles

## Import necessary modules and environment variables

In [1]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import sqlalchemy_utils
import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
load_dotenv()

True

## Let's also go ahead and create our engine and database

In [2]:
database_uri = f"{os.environ['DATABASE_BASE_URI']}movies"
engine = sqlalchemy.create_engine(database_uri)

In [3]:
try:
  sqlalchemy_utils.create_database(database_uri)
except:
  pass
sqlalchemy_utils.database_exists(database_uri)

True

## Load the full flat file

In [4]:
df = pd.read_csv("./data/tmdb_results_combined.csv.gz")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88787 entries, 0 to 88786
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           88787 non-null  object 
 1   titleType        88787 non-null  object 
 2   primaryTitle     88787 non-null  object 
 3   originalTitle    88787 non-null  object 
 4   isAdult          88787 non-null  int64  
 5   startYear        88787 non-null  int64  
 6   endYear          0 non-null      float64
 7   runtimeMinutes   88787 non-null  int64  
 8   genres           88787 non-null  object 
 9   averageRating    74859 non-null  float64
 10  numVotes         74859 non-null  float64
 11  ordering         88787 non-null  int64  
 12  title            88787 non-null  object 
 13  region           88787 non-null  object 
 14  language         891 non-null    object 
 15  types            82484 non-null  object 
 16  attributes       4147 non-null   object 
 17  isOriginalTi

## Filter out the akas

In [5]:
no_akas = df.drop_duplicates(subset=["primaryTitle"]).copy()
no_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71787 entries, 0 to 88785
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tconst           71787 non-null  object 
 1   titleType        71787 non-null  object 
 2   primaryTitle     71787 non-null  object 
 3   originalTitle    71787 non-null  object 
 4   isAdult          71787 non-null  int64  
 5   startYear        71787 non-null  int64  
 6   endYear          0 non-null      float64
 7   runtimeMinutes   71787 non-null  int64  
 8   genres           71787 non-null  object 
 9   averageRating    59543 non-null  float64
 10  numVotes         59543 non-null  float64
 11  ordering         71787 non-null  int64  
 12  title            71787 non-null  object 
 13  region           71787 non-null  object 
 14  language         411 non-null    object 
 15  types            67764 non-null  object 
 16  attributes       2383 non-null   object 
 17  isOriginalTi

## Just ensuring there are no duplicates in our data

In [6]:
no_akas["tconst"].duplicated().sum()

0

## Making the tables

### Lets start with the movies table

In [7]:
movies = no_akas[["tconst","primaryTitle","startYear","runtimeMinutes"]]
movies.columns = ["titleId", *movies.columns[1:]]
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71787 entries, 0 to 88785
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   titleId         71787 non-null  object
 1   primaryTitle    71787 non-null  object
 2   startYear       71787 non-null  int64 
 3   runtimeMinutes  71787 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.7+ MB


#### Just a little correction to ensure the table is saved with the string id

In [8]:
movie_schema= {
  "titleId":sqlalchemy.types.String(255),
  "primaryTitle":sqlalchemy.types.Text(),
  "startYear":sqlalchemy.types.Integer(),
  "runtimeMinutes":sqlalchemy.types.Integer()
}
movies.to_sql("title_basics", engine,dtype=movie_schema,if_exists="replace", index=False)
pd.read_sql_query("""SHOW TABLES;""", engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,tmdb_data


In [9]:
engine.execute("""ALTER TABLE title_basics ADD PRIMARY KEY (`titleId`);""")

In [10]:
pd.read_sql_query("""SELECT * FROM title_basics LIMIT 1;""", engine)

,titleId,primaryTitle,startYear,runtimeMinutes
0,tt0035423,Kate & Leopold,2001,118


### Now the TMDB data table

In [11]:
tmdb = no_akas[["budget","revenue","rating","imdb_id"]]
tmdb.columns = [*tmdb.columns[:-1],"titleId"]
tmdb.index.rename("tmdbId", inplace=True)
tmdb.head()

,budget,revenue,rating,titleId
tmdbId,,,,
0,48000000.0,76019048.0,PG-13,tt0035423
2,0.0,0.0,NaN,tt0062336
3,12000000.0,0.0,R,tt0069049
4,350000.0,0.0,NaN,tt0088751
6,187.0,0.0,NaN,tt0094859


In [12]:
tmdb.to_sql("tmdb_data", engine, if_exists="replace")
pd.read_sql_query("""SHOW TABLES;""", engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,tmdb_data


In [13]:
engine.execute("""ALTER TABLE tmdb_data ADD PRIMARY KEY (`tmdbId`);""")

In [14]:
pd.read_sql_query("""SELECT * FROM tmdb_data LIMIT 1;""", engine)

,tmdbId,budget,revenue,rating,titleId
0,0,48000000.0,76019048.0,PG-13,tt0035423


### Genres is going to be a bit more diificult

#### First let's get all the tconst values with each of their genres

In [15]:
no_akas["splitGenres"] = no_akas["genres"].str.split(",")
genres = no_akas.explode("splitGenres")[["tconst","splitGenres"]]
genres.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 134264 entries, 0 to 88785
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   tconst       134264 non-null  object
 1   splitGenres  134264 non-null  object
dtypes: object(2)
memory usage: 3.1+ MB


#### Then assign each unique genre to an id

In [16]:
unique_genres = sorted(genres["splitGenres"].unique())
genre_map = dict(zip(unique_genres,range(len(unique_genres))))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Short': 20,
 'Sport': 21,
 'Talk-Show': 22,
 'Thriller': 23,
 'War': 24,
 'Western': 25}

#### Update the DataFrame with the id's

In [17]:
genres["genreId"] = genres["splitGenres"].map(genre_map)
genres.drop("splitGenres", axis = 1, inplace = True)
genres.columns = ["titleId","genreId"]
genres.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134264 entries, 0 to 88785
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   titleId  134264 non-null  object
 1   genreId  134264 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


#### Add this many to many connector table to sql

In [18]:
genres.to_sql("title_genres",engine, index=False, if_exists="replace")
pd.read_sql_query("""SHOW TABLES;""", engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,tmdb_data


#### Make the genre table for the many to many to connect to

In [19]:
genre_table = pd.DataFrame.from_dict({"genreId":genre_map.values(),"genreName":genre_map.keys()})
genre_table.head()

,genreId,genreName
0,0,Action
1,1,Adult
2,2,Adventure
3,3,Animation
4,4,Biography


#### Add that table to sql

In [20]:
genre_table.to_sql("genres", engine, index=False, if_exists="replace")
pd.read_sql_query("""SHOW TABLES;""", engine)

,Tables_in_movies
0,genres
1,title_basics
2,title_genres
3,tmdb_data


In [21]:
engine.execute("""ALTER TABLE genres ADD PRIMARY KEY (`genreId`);""")

### Now just to add foreign keys and look at our database

#### First we need to set all the titleId columns to the same length

In [25]:
for table in ["title_basics","title_genres","tmdb_data"]:
  engine.execute(f"""ALTER TABLE {table} MODIFY COLUMN titleId VARCHAR(255);""")

#### Now to add the foreign keys

In [26]:
engine.execute("""
ALTER TABLE tmdb_data
ADD FOREIGN KEY (titleId)
REFERENCES title_basics (titleId);
"""
)

In [27]:
engine.execute("""
ALTER TABLE title_genres
ADD FOREIGN KEY (titleId)
REFERENCES title_basics (titleId);
"""
)

In [28]:
engine.execute("""
ALTER TABLE title_genres
ADD FOREIGN KEY (genreId)
REFERENCES genres (genreId);
"""
)

### Now lets make an ERB in MySQL Workbench and view it

![MoviesERD](./img/MoviesERD.png)

## Everything looks good!